In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)


/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
learning_rate = 0.0005   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
train_keep_prob = 0.7
max_len = 200           # sequence 단어 수 제한
max_vocab = 40000       # maximum 단어 개수
emb_dim = 128            # 단어 embedding dimension
hidden_dim = 128        # RNN hidden dim
train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
hypothesis_summary = []
##################################################
from ensemble_loader import movie_data
data = movie_data(max_vocab=max_vocab, max_len=max_len)

train_total_batch = int(data.train_size / BATCH_SIZE)
test_total_batch = int(data.test_size / BATCH_SIZE)

from genre_ensemble_model_v3 import Model
model = Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

Using TensorFlow backend.


x_train size 4800
y_train size 4800
x_test  size 1200
y_test  size 1200
x_ids_train  size 4800
x_len_train  size 4800
x_ids_test  size 1200
x_len_test  size 1200
(?, 128)


In [3]:
model.restore(sess)

 - Restoring variables...
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from models/model
 * model restored 


In [31]:
test_file = []
#input_files = ['avengers', 'gonsiam', 'trump']
input_files = ['ant','witch']


test_y = [0,0]

for i in input_files:
    test_file.append(i+'.jpg')

test_file 

['ant.jpg', 'witch.jpg']

In [32]:
test_x_img, test_ids, test_length= data.init_data_test(test_file, '../dataset/poster_test/','../dataset/poster_test/')
#test_x_img, test_ids, test_length= data.init_data_test(test_file)
hypothesys = sess.run(model.genre_prob,
    feed_dict={model.x_image: test_x_img, model.x_ids: test_ids, model.x_len: test_length,
                              model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})

In [33]:
hypothesys_ = hypothesys.tolist()

In [34]:
for file, prob_list in zip(test_file,hypothesys_):
    print(file)
    for i, prob in enumerate(prob_list):
        #print(prob)
        print(data.labelid_2_genre[i], '{:.2f}'.format(prob * 100), "%")
    print('-' * 100)

ant.jpg
Romance 82.77 %
Horror 1.72 %
Adventure 13.97 %
Documentary 1.53 %
----------------------------------------------------------------------------------------------------
witch.jpg
Romance 11.63 %
Horror 21.45 %
Adventure 59.79 %
Documentary 7.13 %
----------------------------------------------------------------------------------------------------


'Horror'

In [8]:
def test_model():
    test_loss, test_acc = 0, 0

    
    mid, test_x_img, test_ids, test_length, test_y = data.get_test(100)

    loss, acc, hy = sess.run([model.loss, model.accuracy, model.genre_prob],
                              feed_dict={model.x_image: test_x_img, model.x_ids: test_ids, model.x_len: test_length,
                              model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})
    
    hypothesys_ = hy.tolist()
    #print(hypothesys_)
    for m, prob_list in zip(mid,hypothesys_):
        print(m)
        for i, prob in enumerate(prob_list):
            print(i, '{:.2f}'.format(prob * 100), "%")
        print('-' * 100)
    #print(" * test loss: {:.3f} | test acc: {:.3f}\n".format(test_loss / test_cnt, sess.run(model.accuracy, feed_dict={model.x_image: test_x, model.y_label: test_label})))



In [13]:
test_model()

1712159
0 0.00 %
1 99.34 %
2 0.66 %
3 0.00 %
----------------------------------------------------------------------------------------------------
86349
0 0.00 %
1 96.69 %
2 3.31 %
3 0.00 %
----------------------------------------------------------------------------------------------------
1550312
0 99.24 %
1 0.45 %
2 0.12 %
3 0.19 %
----------------------------------------------------------------------------------------------------
3529920
0 0.08 %
1 0.26 %
2 35.04 %
3 64.62 %
----------------------------------------------------------------------------------------------------
86232
0 9.83 %
1 0.14 %
2 89.84 %
3 0.19 %
----------------------------------------------------------------------------------------------------
2822742
0 99.70 %
1 0.23 %
2 0.04 %
3 0.02 %
----------------------------------------------------------------------------------------------------
1068961
0 11.60 %
1 88.21 %
2 0.19 %
3 0.01 %
---------------------------------------------------------------------------------

In [12]:
#1613763
#0 39.07 %
#1 3.80 %
#2 14.78 %
#3 42.34 %

#117427
#0 31.71 %
#1 29.54 %
#2 38.72 %
#3 0.03 %


data.movieid_2_genre[117427]

'Adventure'